In [1]:
import json
import numpy as np
import threading
import time
import pandas as pd
from pcomp_utils.kafka_producer_utils import KafkaProducerHandler
from pcomp_utils.kafka_consumer_utils import KafkaConsumerHandler
from pcomp_utils.activation_functions import ACTIVATIONS, relu, softmax
from pcomp_utils.redis_utils import RedisHandler

# Kafka Configuration
KAFKA_BROKER = 'kafka:9092'

class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = ACTIVATIONS.get(activation, relu)
        self.consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}', KAFKA_BROKER, partition=self.neuron_id)
        self.producer = KafkaProducerHandler(KAFKA_BROKER)
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0)

    def fetch_input(self, image_id):
        return self.redis_handler.get("initial_data") if self.layer_id == 'layer_0' else self.redis_handler.get(f'layer_{int(self.layer_id[-1]) - 1}:{image_id}')

    def run(self):
        while True:
            for message in self.consumer.consume():
                if message.value.get('layer') == self.layer_id:
                    image_id = message.value.get('image_id')
                    redis_key = f"{self.layer_id}:neuron_{self.neuron_id}:{image_id}"
    
                    # if self.redis_handler.get(redis_key) is not None:
                    #     continue
    
                    input_data = self.fetch_input(image_id)
                    if self.layer_id == "layer_1":
                        print(input_data)
                    if input_data is not None:
                        output = self.process_data(input_data)
                        #redis_key = f"{self.layer_id}:neuron_{self.neuron_id}:{image_id}"
                        self.redis_handler.set(redis_key, output)
                        #self.redis_handler.set(processed_key, True)
    
                        print(f"✅ Neuron {self.neuron_id} in {self.layer_id} processed and stored data for image {image_id}")
                        self.producer.send(f'layer-{self.layer_id_num}-complete', {'neuron_id': self.neuron_id, 'image_id': image_id}, self.layer_id_num)
                break
            time.sleep(1)

    def process_data(self, inputs):
        return self.activation_func(np.dot(inputs, self.weights) + self.bias)

class LayerCoordinator(threading.Thread):
    def __init__(self, layer_id, neuron_count, is_final_layer=False):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_count = neuron_count
        self.is_final_layer = is_final_layer
        self.completed_neurons = set()  # TODO: make a dictionary with image_id as the keys
        self.consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}-complete', KAFKA_BROKER, self.layer_id_num)
        self.producer = KafkaProducerHandler(KAFKA_BROKER)
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0)

    def run(self):
        while True:
            for message in self.consumer.consume():
                image_id = message.value.get('image_id')
                neuron_id = message.value.get('neuron_id')
                key = f"{image_id}:{neuron_id}"

                if key not in self.completed_neurons:
                    self.completed_neurons.add(key)

                if len(self.completed_neurons) == self.neuron_count:
                    self.aggregate_neuron_outputs(image_id)
                    self.completed_neurons.clear()
                    if not self.is_final_layer:
                        self.activate_next_layer(image_id)
            time.sleep(1)

    def aggregate_neuron_outputs(self, image_id):
        # Initialize the aggregation with zeros to ensure the correct structure
        outputs = np.zeros(self.neuron_count, dtype=np.float32)
    
        for neuron_id in range(self.neuron_count):
            data = self.redis_handler.get(f"{self.layer_id}:neuron_{neuron_id}:{image_id}")
            if data is not None:
                outputs[neuron_id] = data  # Place the neuron's output in the correct position

        print(outputs)
    
        self.redis_handler.set(f"{self.layer_id}:{image_id}", outputs)
        print(f"📝 Aggregated data for {self.layer_id} and image {image_id} stored in Redis.")
    
        if self.is_final_layer:
            prediction = int(np.argmax(outputs))
            self.redis_handler.hset('predictions', image_id, prediction)
            print(f"🎯 Final Prediction for image {image_id}: {prediction}")
    
    def activate_next_layer(self, image_id):
        next_layer = f'layer_{self.layer_id_num + 1}'
        self.producer.send('activate-layer', {'layer': next_layer, 'image_id': image_id}, self.layer_id_num + 1)
        print(f"🚀 Activating next layer: {next_layer} for image {image_id}")
            

class Layer(threading.Thread):
    def __init__(self, layer_id, neuron_count):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.neuron_count = neuron_count
        self.producer = KafkaProducerHandler(KAFKA_BROKER)
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.consumer = KafkaConsumerHandler('activate-layer', KAFKA_BROKER, self.layer_id_num)

    def run(self):
        while True:
            for message in self.consumer.consume():
                if message.value.get('layer') == self.layer_id:
                    image_id = message.value.get('image_id')
                    self.activate_neurons(image_id)
            time.sleep(1)

    def activate_neurons(self, image_id):
        for neuron_id in range(self.neuron_count):
            self.producer.send(f'layer-{self.layer_id_num}', {'layer': self.layer_id, 'image_id': image_id}, neuron_id)
            #print(f"✅ Activating Neuron {neuron_id} in {self.layer_id} for image {image_id}")

def store_initial_input_data(image_np):
    redis_handler = RedisHandler('host.docker.internal', 6379, 0)
    redis_handler.set("initial_data", image_np)
    print(f"📥 Initial input data stored in Redis.")

def activate_network(image_id):
    producer = KafkaProducerHandler(KAFKA_BROKER)
    producer.send('activate-layer', {'layer': 'layer_0', 'image_id': image_id}, 0)
    print(f"🚀 Initial activation sent to activate-layer for layer_0 and image {image_id}")
    producer.close()

# Load network and dataset
data = json.load(open("node_based_model.json"))
df = pd.read_csv('data/mnist.csv').head(2)

neurons = []
layers = []
coordinators = []

for layer_name, layer_info in data.items():
    neurons += [Neuron(layer_id=layer_name, neuron_id=i, weights=node['weights'], bias=node['biases'], activation=node['activation']) for i, node in enumerate(layer_info['nodes'])]
    layers.append(Layer(layer_id=layer_name, neuron_count=len(layer_info['nodes'])))
    coordinators.append(LayerCoordinator(layer_id=layer_name, neuron_count=len(layer_info['nodes']), is_final_layer=(layer_name == list(data.keys())[-1])))

# Start all threads
for thread in neurons + layers + coordinators:
    thread.start()

In [ ]:
# Send images one by one with a 40-second delay
for idx, row in df.iterrows():
    image_np = row.iloc[:-1].values.astype(np.float32)
    store_initial_input_data(image_np)
    activate_network(idx)
    time.sleep(60)

# Wait for all threads to complete
for thread in neurons + layers + coordinators:
    thread.join()

📥 Initial input data stored in Redis.
🚀 Initial activation sent to activate-layer for layer_0 and image 0
✅ Neuron 0 in layer_0 processed and stored data for image 0
✅ Neuron 1 in layer_0 processed and stored data for image 0
✅ Neuron 2 in layer_0 processed and stored data for image 0
✅ Neuron 3 in layer_0 processed and stored data for image 0
✅ Neuron 5 in layer_0 processed and stored data for image 0
✅ Neuron 4 in layer_0 processed and stored data for image 0
✅ Neuron 6 in layer_0 processed and stored data for image 0
✅ Neuron 7 in layer_0 processed and stored data for image 0
✅ Neuron 8 in layer_0 processed and stored data for image 0
✅ Neuron 9 in layer_0 processed and stored data for image 0
✅ Neuron 10 in layer_0 processed and stored data for image 0
✅ Neuron 11 in layer_0 processed and stored data for image 0
✅ Neuron 13 in layer_0 processed and stored data for image 0
✅ Neuron 12 in layer_0 processed and stored data for image 0
✅ Neuron 14 in layer_0 processed and stored data f

/tmp/ipykernel_30/1765709510.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[neuron_id] = data  # Place the neuron's output in the correct position


✅ Neuron 3 in layer_1 processed and stored data for image 0
✅ Neuron 2 in layer_1 processed and stored data for image 0
[ 0.          1.0285566   2.231112    0.          3.526267    0.
  3.7727046   0.          4.321643    0.          0.          0.
  3.2751155   0.          0.          7.071286    0.          0.
  7.414502    0.          1.9019663   0.          6.628729    0.
  2.872025    0.          2.13101     0.          0.          0.
  7.1538033   0.          4.786089    0.          0.          0.
  0.          0.          0.          2.4277217   0.          4.4204826
  0.          0.          0.          0.          0.02829104  0.
  0.          0.          6.462321    0.          0.          0.
  0.          1.9162726   0.          0.          0.          5.6754904
  4.047011    0.          3.015768    3.8851745  10.017956    4.618611
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 10.199376    4